In [1]:
import streamlit as st
from langchain_groq import ChatGroq
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain.chains import LLMChain
from langchain import PromptTemplate

import os

In [3]:
from dotenv import load_dotenv
load_dotenv()

True

In [7]:
groq_api_key = os.getenv("GROQ_API_KEY")
from langchain_groq import ChatGroq
llm = ChatGroq(api_key=groq_api_key,model="llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000026F5BBBA350>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000026F5BBE8580>, model_name='llama3-8b-8192', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [6]:
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("apjspeech.pdf")
docs = loader.load()
docs

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowin

In [8]:
from langchain.chains import LLMChain
from langchain import PromptTemplate

In [9]:
template=""" Write a concise and short summary of the following speech,
Speech :{text}

 """
prompt=PromptTemplate(input_variables=['text'],
                      template=template)
prompt

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template=' Write a concise and short summary of the following speech,\nSpeech :{text}\n\n ')

In [12]:
from langchain.chains.summarize import load_summarize_chain
chain=load_summarize_chain(llm,chain_type='stuff',prompt=prompt,verbose=True)
output_summary=chain.run(docs)
output_summary

C:\Users\Pramod Vepada\AppData\Local\Temp\ipykernel_20872\2361406257.py:3: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  output_summary=chain.run(docs)




> Entering new StuffDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise and short summary of the following speech,
Speech :A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online int

'Here is a concise and short summary of the departing speech by Dr. A.P.J. Abdul Kalam:\n\nDr. Kalam reflects on his five years as President of India, expressing gratitude to the people for their support and cooperation. He highlights 10 key messages that he has learned during his tenure, including:\n\n1. Accelerating development, led by the aspirations of the youth.\n2. Empowering villages and rural communities.\n3. Mobilizing rural core competencies for competitiveness.\n4. Fostering agricultural growth through "Seed to Food".\n5. Defeating problems and succeeding through partnerships.\n6. Overcoming calamities through courage and resilience.\n7. Ensuring connectivity for societal transformation.\n8. Defending the nation through the bravery of its armed forces.\n9. Empowering the youth to become enlightened citizens.\n10. Working towards a Developed India 2020.\n\nDr. Kalam emphasizes the importance of preserving India\'s rich cultural heritage and traditions, while also fostering ec

In [13]:
text_splitters = RecursiveCharacterTextSplitter(chunk_size = 2000,chunk_overlap = 200)
final_docs = text_splitters.split_documents(docs)
final_docs

[Document(metadata={'source': 'apjspeech.pdf', 'page': 0}, page_content='A P J Abdul Kalam Departing speech \n \n \nFriends, I am delighted to address you all, in the country and those living abroad, after \nworking with you and completing five beautiful and eventful years in Rashtrapati \nBhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I \nenjoyed every minute of my tenure enriched by the wonderful assoc iation from each one \nof you, hailing from different walks of life, be it politics, sci ence and technology, \nacademics, arts, literature, business, judiciary, administration, local bodies, farming, \nhome makers, special children, media and above all from the youth and st udent \ncommunity who are the future wealth of our country. During my intera ction at \nRashtrapati Bhavan in Delhi and at every state and union territor y as well as through my \nonline interactions, I have many unique experiences to share with you, which signify the \nfollowin

In [14]:
chunks_prompt="""
Please summarize the below speech:
Speech:`{text}'
Summary:
"""
map_prompt_template=PromptTemplate(input_variables=['text'],
                                    template=chunks_prompt)

In [ ]:
final_prompt='''
Provide the final summary of the entire speech with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
Speech:{text}

'''
final_prompt_template=PromptTemplate(input_variables=['text'],template=final_prompt)
final_prompt_template

PromptTemplate(input_variables=['text'], input_types={}, partial_variables={}, template='\nProvide the final summary of the entire speech with these important points.\nAdd a Motivation Title,Start the precise summary with an introduction and provide the summary in number \npoints for the speech.\nSpeech:{text}\n\n')

In [16]:
summary = load_summarize_chain(llm=llm,chain_type='map_reduce',map_prompt = map_prompt_template,combine_prompt = final_prompt_template,verbose=True)
output = summary.run(final_docs)
output



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Please summarize the below speech:
Speech:`A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have ma

c:\Users\Pramod Vepada\Documents\GENAI Projects\QnAChatbot_MessageHistory\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Pramod Vepada\Documents\GENAI Projects\QnAChatbot_MessageHistory\myenv\lib\site-packages\huggingface_hub\file_download.py:139: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Pramod Vepada\.cache\huggingface\hub\models--gpt2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate 



> Entering new LLMChain chain...
Prompt after formatting:

Provide the final summary of the entire speech with these important points.
Add a Motivation Title,Start the precise summary with an introduction and provide the summary in number 
points for the speech.
Speech:The speech is the departing address of A P J Abdul Kalam, the former President of India, to the nation and those living abroad, after completing five years in Rashtrapati Bhavan. He expresses gratitude to everyone who has supported him during his tenure. He shares 10 important messages that he has learned during his interactions with people from various walks of life, including the youth, villages, rural areas, agriculture, and the nation as a whole. The messages emphasize the need to:

1. Accelerate development and fulfill the aspirations of the youth.
2. Empower villages and rural areas for competitiveness.
3. Mobilize rural core competence for national growth.
4. Focus on agriculture and food production.
5. Overcome

"**Motivation Title:** Empowering India: A Vision for a Developed Nation\n\n**Summary:**\n\nHere are the 10 important messages that A P J Abdul Kalam, the former President of India, shared with the nation and those living abroad:\n\n1. **Accelerate development and fulfill the aspirations of the youth.**\n2. **Empower villages and rural areas for competitiveness.**\n3. **Mobilize rural core competence for national growth.**\n4. **Focus on agriculture and food production.**\n5. **Overcome problems and succeed through determination.**\n6. **Partner with others to overcome challenges.**\n7. **Show courage in the face of calamities.**\n8. **Leverage connectivity for societal transformation.**\n9. **Defend the nation and its pride.**\n10. **Launch a youth movement to achieve Developed India 2020.**\n\nKey points:\n\n* Empowering villages and youth is crucial for achieving development goals.\n* Connectivity and partnership are essential for societal transformation.\n* India needs to focus on 

In [17]:
refine_summary = load_summarize_chain(llm=llm,
                                      chain_type="refine",
                                      verbose=True)
Output_refinedSummary = refine_summary.run(final_docs)
Output_refinedSummary



> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
Write a concise summary of the following:


"A P J Abdul Kalam Departing speech 
 
 
Friends, I am delighted to address you all, in the country and those living abroad, after 
working with you and completing five beautiful and eventful years in Rashtrapati 
Bhavan. Today, it is indeed a thanks giving occasion. I would like to narr ate, how I 
enjoyed every minute of my tenure enriched by the wonderful assoc iation from each one 
of you, hailing from different walks of life, be it politics, sci ence and technology, 
academics, arts, literature, business, judiciary, administration, local bodies, farming, 
home makers, special children, media and above all from the youth and st udent 
community who are the future wealth of our country. During my intera ction at 
Rashtrapati Bhavan in Delhi and at every state and union territor y as well as through my 
online interactions, I have many

"Based on the new context, I have refined the original summary to provide a more comprehensive overview of President A.P.J. Abdul Kalam's departing speech. Here is the refined summary:\n\nPresident A.P.J. Abdul Kalam's departing speech emphasizes the importance of empowering India's youth and villages to accelerate the country's development. He highlights the potential for transformation in India's 600,000 villages, citing his own experience in Nagaland where he saw a prosperous village with empowered council members and quality roads.\n\nKalam also showcases the success of Periyar PURA (Providing Urban Amenities to Rural Areas) initiative, which has led to sustainable economic development in rural areas. He emphasizes the need for more such PURA initiatives, suggesting that the whole country needs around 7000 PURA to bridge the rural-urban divide.\n\nFurthermore, Kalam emphasizes the importance of preserving India's rich and diverse culture and civilization for future generations, and